# Compute reduced alphabet 

In [1]:
import pandas as pd
import pyrepseq as prs
import numpy as np
import itertools
import matplotlib.pyplot as plt

/Users/james/Documents/Projects/CI_CRUK_2025/venv/lib/python3.12/site-packages/logomaker/../tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import data

In [3]:
#DATA_PATH = '/home/jhenderson/Documents/Projects/data_sets/tcr_sequences/CI_CRUK_datathon/'
DATA_PATH = '/Users/james/Documents/Projects/data_sets/tcr_sequences/CI_CRUK_datathon/'

train = pd.read_csv(DATA_PATH + 'processed_data/train_fixed_lengths.csv')
train = train.dropna(subset=['CDR3B', 'epitope', 'Assays']).reset_index(drop=True)
train = train[train['Assays'].str.contains('mer')]
train['epitope'] = train['epitope'].apply(lambda x: x[2:7])
train['CDR3B'] = train['CDR3B'].apply(lambda x: x[4:14])
aminoacids = 'ACDEFGHIKLMNPQRSTVWY'

## Construct all possible reduced epitopes

In [4]:
letters = ['0', '1']
possible_reduced_epitopes = [''.join(i) for i in itertools.product(letters, repeat = 5)]
reduced_epitope_letter_counts = {epitope : [epitope.count('0'), epitope.count('1')] for epitope in possible_reduced_epitopes}

## Translation evaluation

In [5]:
def translate_df(df, translation, columns_to_translate):
    
    df = df.copy()
    translation_table = str.maketrans(translation)
    for column in columns_to_translate:
        df['translation_'+column] = df[column].apply(lambda x: x.translate(translation_table))
    return df

In [6]:
def compute_epitope_prior(translation):
    
    alphabet = list(translation.values()) 
    counts = {'0':alphabet.count('0'), '1':alphabet.count('1')}
    priors = pd.Series(index=possible_reduced_epitopes)
    for epitope in possible_reduced_epitopes:
        priors[epitope] = ((counts['0']/20)**(reduced_epitope_letter_counts[epitope][0]))*((counts['1']/20)**(reduced_epitope_letter_counts[epitope][1]))
         
    return priors

In [7]:
def pc_conditional(df, feature, grouping, group_weights):
    
    df = df.groupby(grouping).filter(lambda x: len(x) > 1)
    conditional_pcs = df.groupby(grouping).apply(lambda x: prs.pc(x[feature]), include_groups=False)
    adjusted_group_weights = (group_weights**2)/np.sum(group_weights**2)
    
    return np.sum(adjusted_group_weights * conditional_pcs)

In [8]:
def pc_cross(df, feature, grouping, group_weights):
    
    groups = sorted(list(df.groupby(grouping)))
    pc_group = np.sum(group_weights**2)
    data = []
    for ((name1, d1)), (name2, d2) in itertools.combinations(groups, 2):
        pc_cross_group = prs.pc(d1[feature], d2[feature])
        weight = 2*(group_weights[name1] * group_weights[name2]) / (1-pc_group)
        data.append(weight * pc_cross_group)      
    data = np.array(data)
    return np.sum(data)

In [9]:
def estimate_background_pc(df, feature, grouping, epitope_weights):
    
    pc_within = pc_conditional(df, feature, grouping, epitope_weights)
    pc_across = pc_cross(df, feature, grouping, epitope_weights)
    pc_epitope = np.sum(epitope_weights**2)
     
    return pc_epitope*pc_within + (1-pc_epitope)*pc_across

In [10]:
def evaluate_translation(translation, df):
     
    df = translate_df(df, translation, ['CDR3B', 'epitope'])
    epitope_weights = compute_epitope_prior(translation)
        
    background_beta_entropy = -np.log2(estimate_background_pc(df, 'translation_CDR3B', 'translation_epitope', epitope_weights))
    specific_beta_entropy = -np.log2(pc_conditional(df, 'translation_CDR3B', 'translation_epitope', epitope_weights))
     
    return background_beta_entropy - specific_beta_entropy

In [11]:
def make_translation_from_list(aa_to_be_one):
    
    return {aa: '1' if aa in aa_to_be_one else '0' for aa in aminoacids}

In [12]:
def greedy_algorithm(df):
    
    best_amino_acids_to_be_one = []
    best_score = -np.inf
    for i in range(20):
        best_amino_acid_to_be_one = ""
        best_local_score = -np.inf
        for aa in aminoacids:
            if aa not in best_amino_acids_to_be_one:
                list_to_try = best_amino_acids_to_be_one.copy()
                list_to_try.append(aa)
                translation = make_translation_from_list(list_to_try)
                score = evaluate_translation(translation, df)
                
                if score > best_local_score:
                    best_amino_acid_to_be_one = aa
                    best_local_score = score
        
        if best_local_score <= best_score:
            print("Locally optimal set found")
            return best_amino_acids_to_be_one, best_score
        
        print(f"Improvement found, new score: {best_local_score:.1f} bits")
        best_score = best_local_score
        best_amino_acids_to_be_one.append(best_amino_acid_to_be_one)
            
    return best_amino_acids_to_be_one, best_score

## Training the representation

In [13]:
best, score = greedy_algorithm(train)

Improvement found, new score: 0.1 bits
Improvement found, new score: 0.2 bits
Improvement found, new score: 0.4 bits
Improvement found, new score: 0.7 bits
Improvement found, new score: 1.6 bits
Improvement found, new score: 2.5 bits
Improvement found, new score: 2.9 bits
Improvement found, new score: 3.2 bits
Improvement found, new score: 3.4 bits
Locally optimal set found


In [15]:
best, [aa for aa in aminoacids if aa not in best], score

(['R', 'E', 'Q', 'G', 'S', 'Y', 'C', 'K', 'H'],
 ['A', 'D', 'F', 'I', 'L', 'M', 'N', 'P', 'T', 'V', 'W'],
 np.float64(3.437883625079337))

In [16]:
best_translation = make_translation_from_list(best)

## Save the data

In [19]:
pd.DataFrame(best_translation, index=['Letter']).to_csv(DATA_PATH + "/reduction_outputs/alphabet.csv")